In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install evaluate
!pip install jiwer
!pip install gradio
!pip install  bitsandbytes datasets accelerate
!pip install  peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 90.4 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.6 MB/s eta 0:00:00


In [3]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
from datasets import load_dataset, Audio
import evaluate

# Load model and processor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2").to(device)

# Load dataset
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test",trust_remote_code=True)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Load WER metric
wer_metric = evaluate.load("wer")

# Define function to compute WER and print transcriptions
def compute_wer(batch):
    inputs = processor(batch["audio"]["array"], sampling_rate=16000, return_tensors="pt").input_features.to(device)
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="hi", task="transcribe")
    predicted_ids = model.generate(inputs, forced_decoder_ids=forced_decoder_ids)
    
    # Decode the output without skipping special tokens (so no normalization is applied)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)[0]
    
    return {"transcription": transcription, "ground_truth": batch["sentence"]}

# Map dataset to get results
results = dataset.map(compute_wer)

# Print transcriptions and ground truths
for i in range(len(results)):
    print(f"Ground Truth: {results[i]['ground_truth']}")
    print(f"Transcription: {results[i]['transcription']}\n")

# Calculate and print WER
wer = wer_metric.compute(predictions=results["transcription"], references=results["ground_truth"])
print(f"Word Error Rate (WER): {wer}")


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 680it [00:00, 96695.37it/s]


Map:   0%|          | 0/2894 [00:00<?, ? examples/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Ground Truth: वाराणसी हिंसा: पुलिस की भूमिका संदिग्ध, वायरल हुई तस्वीरें
Transcription: <|startoftranscript|><|hi|><|transcribe|><|notimestamps|> वारानुसी हिंसा पुलीस की भूमी का संदिक्त बाइरिल हुई तस्वीरे

Ground Truth: Box office का नया हीरो बना हॉरर, 'लैला मजनू', 'पलटन' पीछे
Transcription: <|startoftranscript|><|hi|><|transcribe|><|notimestamps|> Box Office ka naya Hero bana Horror Laila Majnu Burton Peeche

Ground Truth: Opinion: विराट कोहली के लिए बेहतरीन मौका
Transcription: <|startoftranscript|><|hi|><|transcribe|><|notimestamps|> वीराट कोली के लिए बेहतरिन मौता

Ground Truth: छेड़छाड़ का विरोध कर रहे छात्रों को पुलिस ने पीटा और जेल में डाला
Transcription: <|startoftranscript|><|hi|><|transcribe|><|notimestamps|> छेड़ चारता विरोध कर रहे छात्रों को पूलिस ने पीटा और जेल में डाला।

Ground Truth: तुम्हारे बहुत सारे दुशमन हैं।
Transcription: <|startoftranscript|><|hi|><|transcribe|><|notimestamps|> तुम्हारे बहुत सारे दुष्मन हैं।

Ground Truth: आप बना सकते हैं।
Transcription: <|startoftr